In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from neuralforecast.core import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import MAE, MSE, QuantileLoss
import torch.nn as nn
import warnings
import os

# Filter specific warnings
warnings.filterwarnings("ignore", message="val_check_steps is greater than max_steps")
warnings.filterwarnings("ignore", message="The 'val_dataloader' does not have many workers")
warnings.filterwarnings("ignore", message="The 'train_dataloader' does not have many workers")
warnings.filterwarnings("ignore", message="The number of training batches")
warnings.filterwarnings("ignore", message="`Trainer.fit` stopped: `max_steps=16` reached.")
warnings.filterwarnings("ignore", message="Trying to infer the `batch_size` from an ambiguous collection")

import logging
logging.getLogger("pytorch_lightning").setLevel(logging.CRITICAL)

import pytorch_lightning as pl
# Trainer configuration
trainer = pl.Trainer(logger=False)
    

In [2]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon
data, X_df, S_df = LongHorizon.load(directory='./data', group='Weather')

# Function to apply Min-Max normalization
def min_max_normalize(group):
    min_y = group['y'].min()
    max_y = group['y'].max()
    group['y'] = (group['y'] - min_y) / (max_y - min_y)
    # date_range = pd.date_range(start='200-01-01', periods=len(group), freq='2W')
    # group['ds'] = date_range
    group['ds'] = pd.to_datetime(group['ds'])
    return group

Pre_train_data = data.groupby('unique_id').apply(min_max_normalize)

print(Pre_train_data)

               unique_id                  ds         y
0        H2OC (mmol/mol) 2020-01-01 00:10:00  0.151515
1        H2OC (mmol/mol) 2020-01-01 00:20:00  0.149733
2        H2OC (mmol/mol) 2020-01-01 00:30:00  0.148841
3        H2OC (mmol/mol) 2020-01-01 00:40:00  0.146168
4        H2OC (mmol/mol) 2020-01-01 00:50:00  0.148396
...                  ...                 ...       ...
1106590         wv (m/s) 2020-12-31 23:20:00  0.998698
1106591         wv (m/s) 2020-12-31 23:30:00  0.998668
1106592         wv (m/s) 2020-12-31 23:40:00  0.998663
1106593         wv (m/s) 2020-12-31 23:50:00  0.998682
1106594         wv (m/s) 2021-01-01 00:00:00  0.998682

[1106595 rows x 3 columns]


/tmp/ipykernel_803844/1765509857.py:15: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  Pre_train_data = data.groupby('unique_id').apply(min_max_normalize)


In [3]:
# import pandas as pd
# from datasetsforecast.m4 import M4
# data, _,  S_df = M4.load(directory='./data', group='Monthly')
# # data.groupby('unique_id').head(1)


# # Function to split each group
# def split_train_test(data, n):
#     train_frames = []
#     test_frames = []
#     for _, group in data.groupby('unique_id'):
#         train, test = group[:-n], group[-n:]
#         train_frames.append(train)
#         test_frames.append(test)
#     train_df = pd.concat(train_frames)
#     test_df = pd.concat(test_frames)
#     train_df.reset_index(inplace=True, drop=True)
#     test_df.reset_index(inplace=True, drop=True)
#     return train_df, test_df


# # Function to apply Min-Max normalization
# def min_max_normalize(group):
#     min_y = group['y'].min()
#     max_y = group['y'].max()
#     group['y'] = (group['y'] - min_y) / (max_y - min_y)
#     date_range = pd.date_range(start='2000-01-01', periods=len(group), freq='2W')
#     group['ds'] = date_range
#     return group

# # Apply normalization group by group
# # filtered_df = data.groupby('unique_id').filter(lambda x: len(x) >= 650).groupby('unique_id').apply(min_max_normalize)


# # Calculate the size of each group
# group_sizes = data.groupby('unique_id').size()
# # Get the top 500 unique_id with the maximum length
# top_500_unique_ids = group_sizes.nlargest(32000).index
# # Filter the original DataFrame to keep only the rows belonging to these top 500 groups
# filtered_df = data[data['unique_id'].isin(top_500_unique_ids)]
# # Apply normalization group by group
# Pre_train_data = filtered_df.groupby('unique_id').apply(min_max_normalize)

# # _, filtered_df = split_train_test(filtered_df, 2000)


# print(Pre_train_data)

In [4]:
# Define the file name
file_path = 'BasicInputTimeSeries.npy'

# Check if the file exists
if os.path.exists(file_path):
    # Load the .npy file with allow_pickle=True
    time_series_data = np.load(file_path, allow_pickle=True)
    print("Data loaded from file.")
else:
    print("File not found.")


Data loaded from file.


In [5]:
#-------------------------------------------------------------
seed = 100
# Set the random seed for Python's random module
random.seed(seed)
# Set the random seed for NumPy
np.random.seed(seed)
print('seed :',seed)
#-------------------------------------------------------------

    
# Define the column names
columns = ["Year_Mnth_Day", "basin_id", "prcp(mm/day)", "srad(W/m2)", "tmax(C)", "tmin(C)", "vp(Pa)", "QObs(mm/d)"]
# Create the DataFrame
df = pd.DataFrame(time_series_data, columns=columns)

# Deleting the numpy array from memory
del time_series_data

df['ds'] = pd.to_datetime(df['Year_Mnth_Day'])
df['unique_id'] = df['basin_id']
df['y'] = df['prcp(mm/day)'].astype(float)
df.drop(['Year_Mnth_Day', 'basin_id', "prcp(mm/day)", ], axis=1, inplace=True) # "srad(W/m2)", "tmax(C)", "tmin(C)", "vp(Pa)", "QObs(mm/d)"

# for col in df.columns:
#     print(col)
#     if col != 'ds' and col !='unique_id':
#         # Global Normalization
#         scaler = MinMaxScaler(feature_range=(0, 1))
#         df[col] = scaler.fit_transform(df[[col]])


# Sort the DataFrame by 'unique_id' and 'ds' to ensure correct order
df.sort_values(by=['unique_id', 'ds'], inplace=True)

# Function to split each group
def split_train_test(data, n):
    train_frames = []
    test_frames = []
    for _, group in data.groupby('unique_id'):
        train, test = group[:-n], group[-n:]
        train_frames.append(train)
        test_frames.append(test)
    train_df = pd.concat(train_frames)
    test_df = pd.concat(test_frames)
    train_df.reset_index(inplace=True, drop=True)
    test_df.reset_index(inplace=True, drop=True)
    return train_df, test_df

# Function to drop the last n values fron each group
def drop_last_n(data, n):
    train_frames = []
    for _, group in data.groupby('unique_id'):
        train = group[:-n]
        train_frames.append(train)
    train_df = pd.concat(train_frames)
    train_df.reset_index(inplace=True, drop=True)
    return train_df


    
# Set 'ds' as the index if you plan to use time-based indexing
df.set_index('ds', inplace=True, drop=False)

# # Function to calculate EMA for each group
# def calculate_ema(group, span):
#     return group.ewm(span=span, adjust=False).mean()

# # Apply the function to each group for multiple spans
# span_settings = [2, 4, 6, 8, 10, 20, 50]  # Example spans
# for span in span_settings:
#     print(f'EMA_{span}')
#     df[f'EMA_{span}'] = df.groupby('unique_id')['y'].transform(lambda x: calculate_ema(x, span))
    
df.reset_index(inplace=True, drop=True)
   
    
# Number of records to be taken as test data for each unique_id
n = 3000 
n_time_series = 671
len_time_series = 7000
_, df = split_train_test(df, len_time_series)

train_df, test_df = split_train_test(df, n)


print("DataFrame:")
print(df)



seed : 100
DataFrame:
        srad(W/m2) tmax(C) tmin(C)  vp(Pa)  QObs(mm/d)         ds unique_id  \
0           237.53   12.75   -1.37  599.13    0.376714 1989-11-02   1013500   
1            99.59     8.1   -0.31  561.45    0.379961 1989-11-03   1013500   
2           129.07    3.35   -4.01  425.97    0.397282 1989-11-04   1013500   
3           193.49   -0.75   -9.83   303.0    0.405942 1989-11-05   1013500   
4           159.91    3.82   -5.66  425.19    0.422179 1989-11-06   1013500   
...            ...     ...     ...     ...         ...        ...       ...   
4696995     125.72    4.86    0.39  604.28   22.806309 2008-12-27  14400000   
4696996     101.25    6.06    2.83  702.39  115.775145 2008-12-28  14400000   
4696997     140.54    7.87    2.46  703.56  134.606043 2008-12-29  14400000   
4696998     171.69    6.96   -3.14  532.92    54.40037 2008-12-30  14400000   
4696999     174.73    8.15    0.13  599.32   29.920203 2008-12-31  14400000   

              y  
0          

In [6]:


horizon = 1
input_size = 130
models = [PatchTST(h = horizon, input_size=input_size, max_steps=3, learning_rate=1e-4, loss= MSE(), random_seed=1, step_size=1, batch_size=256, scaler_type=None)]
nf = NeuralForecast(models=models, freq='1D', local_scaler_type=None)


for i in range(100):
    print("fit :", i+1)
    nf.fit(Pre_train_data, verbose=False, val_size=0)


for i in range(25):
    print("fit :", i+1)
    nf.fit(train_df, verbose=False, val_size=0)
    
    

[rank: 0] Seed set to 1


fit : 1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

fit : 25


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [7]:

# Create an empty DataFrame to store all predictions
all_preds = pd.DataFrame()
for i in range(n, 0, -horizon):
    print("test :", i)
    _, re_train_df = split_train_test(df, i+input_size)
    data_part = drop_last_n(re_train_df, i)
    preds = nf.predict(data_part)
    preds = preds.reset_index(drop=False)
    all_preds = pd.concat([all_preds, preds], ignore_index=True)
    


test : 3000


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/jrp5td/.local/lib/python3.10/site-packages/neuralforecast/core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


test : 2999


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

/home/jrp5td/.local/lib/python3.10/site-packages/neuralforecast/core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


test : 1


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/jrp5td/.local/lib/python3.10/site-packages/neuralforecast/core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [9]:

all_preds = all_preds.sort_values(by=['unique_id', 'ds'], ascending=[True, True])
results_df = test_df.copy()
results_df['PatchTST-Weather'] = all_preds['PatchTST'].values


In [10]:
test_actual_y = results_df['y']
test_predicted_y = results_df['PatchTST-Weather']
train_actual_y = train_df['y']


index_date_test = results_df[results_df['unique_id'] == 1013500]['ds']
index_date_train = train_df['ds'][:len_time_series-n]

In [11]:

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def normalized_nash_sutcliffe_efficiencySTavg(y_true, y_pred):  # axis 0 space 1 time
    NSE = 1 - np.sum(np.square(y_true - y_pred)) / np.sum(np.square(y_true - np.mean(y_true)))
    return 1 / (2 - NSE)

def mean_squared_error(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def r_squared(y_true, y_pred):
    ss_res = np.sum(np.square(y_true - y_pred))
    ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
    return 1 - (ss_res / ss_tot)

# Assuming your data is organized in results_df and reshaped
Y_ACTUAL = results_df['y'].values.reshape(n_time_series, n)
Y_HAT = results_df['PatchTST-Weather'].values.reshape(n_time_series, n)

MAE = mean_absolute_error(Y_ACTUAL, Y_HAT)
MSE = mean_squared_error(Y_ACTUAL, Y_HAT)
NNSE = normalized_nash_sutcliffe_efficiencySTavg(Y_ACTUAL, Y_HAT)
R2 = r_squared(Y_ACTUAL, Y_HAT)

print("MSE: ", MSE)
print("MAE: ", MAE)
print("NNSE: ", NNSE)
print("R²: ", R2)


MSE:  31.634522115758163
MAE:  2.8184045432727043
NNSE:  0.5955956302297948
R²:  0.3210085009955892


In [12]:
results_df.to_pickle('PatchTST-Weather_prcp.pkl')